Introduction to Microsoft Agent Framework

topics of the SK 2.1:
1. One simple agent - done
2. Dynamic agent instructions with template parameters - irrelevant (??)
3. Multi-Turn Conversations
4. EX1 - Simple Question-Answering Agent
5. Length of chat history in check
6. Function calling and plugins
7. Plugin example for agent
8. What happened in the background - function invocation setup
9. function calling modes
10. EX2 - create your own plugin
11. agent that generates creative content and how streaming works 
12. static error handling

for AF - dynamic agent creation?? profiles and roles?? state management?? workflows??

### Our first Agent

Now, let's build an single agent to get started. First, create a chat client for communicating with Azure OpenAI:

```
AZURE_OPENAI_ENDPOINT - The endpoint it should talk to by default
AZURE_OPENAI_API_KEY - The API Key it should use
AZURE_OPENAI_API_VERSION - Inference API version it should use per default
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME - Model deployment name it should use per default
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME - Embedding deployment name it should use per default
```

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

Then, create the agent, providing instructions and a name for the agent. To run the agent, call the `.run()` method on the agent instance, providing the user input. 

The agent will return a response object, and accessing the .text property provides the text result from the agent.

Please note that we do not have an chat history, so every message we send to the agent is treated as a new conversation.

In [ ]:
simple_agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

response = await simple_agent.run("What's the capital of Latvia?")
print(response.text)


The capital of Latvia is Riga.


### Enabling Multi-Turn Conversations

Agents are stateless and do not maintain any state internally between calls. To have a multi-turn conversation with an agent, you need to create an object to hold the conversation state and pass this object to the agent when running it.

An AgentThread maintains the conversation state and message history for an agent interaction. It can either use a service-managed thread (via service_thread_id) or a local message store (via message_store), but not both.

To initialize an AgentThread, call the `get_new_thread()` method on the agent instance.

You can then pass this thread object to the run and run_stream methods on the agent instance, along with the user input.




In [ ]:
agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

# create a thread on the agent instance:
thread = agent.get_new_thread()

user_messages = [
    "Hello!",
    "Which country has Paris as the capital?",
    "What are its neighbouring countries? Give a short one-liner list, please."
]

# Loop through user messages and maintain context
for user_message in user_messages:
    print("*** User:", user_message)
    
    # get response from the agent, passing the same thread
    response = await agent.run(user_message, thread=thread)
    print("*** Agent:", response.text)

print("-" * 25)

# Print the final conversation history from the thread
messages = await thread.message_store.list_messages()
for msg in messages:
    print(f"[{msg.role}] {msg.text}")


*** User: Hello!
*** Agent: Hello! How can I assist you today?
*** User: Which country has Paris as the capital?
*** Agent: Paris is the capital of France. If you have any more questions about France or anything else, feel free to ask!
*** User: What are its neighbouring countries? Give a short one-liner list, please.
*** Agent: France shares its borders with the following neighboring countries:

1. Belgium
2. Luxembourg
3. Germany
4. Switzerland
5. Italy
6. Monaco
7. Spain
8. Andorra
9. Brazil (via French Guiana)
10. Suriname (via French Guiana)

Let me know if you need more information!
-------------------------
[user] Hello!
[assistant] Hello! How can I assist you today?
[user] Which country has Paris as the capital?
[assistant] Paris is the capital of France. If you have any more questions about France or anything else, feel free to ask!
[user] What are its neighbouring countries? Give a short one-liner list, please.
[assistant] France shares its borders with the following neighbor

### Single agent with multiple conversations

It is possible to have multiple, independent conversations with the same agent instance, by creating multiple `AgentThread` objects. These threads can then be used to maintain separate conversation states for each conversation. The conversations will be fully independent of each other, since the agent does not maintain any state internally.

In [ ]:
# Create two threads for two separate conversations
thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

# First messages for each thread
result1 = await agent.run("Suggest 1 key attraction for Paris trip. Keep it brief.", thread=thread1)
print("Thread 1:", result1.text)

result2 = await agent.run("Suggest 1 key attraction for Tokyo trip. Keep it brief.", thread=thread2)
print("Thread 2:", result2.text)

# Continue each conversation independently
result3 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread1
)
print("Thread 1:", result3.text)

result4 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread2
)
print("Thread 2:", result4.text)


Thread 1: Visit the Eiffel Tower for iconic views of the city and a quintessential Parisian experience.
Thread 2: Visit the Sensō-ji Temple in Asakusa, Tokyo's oldest temple, known for its historic architecture, vibrant shopping street (Nakamise-dori), and cultural significance.
Thread 1: Stroll through the charming streets of Montmartre and enjoy a coffee at a local café.
Thread 2: Take a stroll through the tranquil Shinjuku Gyoen National Garden for a peaceful start to your day.
